In [1]:
# FUNCTIONS:
def univariate_stats(df, roundto=4):

  import pandas as pd
  import numpy as np

  df_results = pd.DataFrame(columns=['dtype', 'count', 'missing', 'unique', 'mode', 'min', 'q1',
                                     'median', 'q3', 'max', 'mean', 'std', 'skew', 'kurt'])

  for col in df:
    dtype = df[col].dtype
    count = df[col].count()
    missing = df[col].isna().sum()
    unique = df[col].nunique()
    try:
      mode = df[col].mode()[0]
    except:
      print(f"Mode can't be calculated for {col}")
      mode = np.nan

    if pd.api.types.is_numeric_dtype(df[col]):
      min = df[col].min()
      q1 = df[col].quantile(.25)
      median = df[col].median()
      q3 = df[col].quantile(.75)
      max = df[col].max()
      mean = df[col].mean()
      std = df[col].std()
      skew = df[col].skew()
      kurt = df[col].kurt()

      df_results.loc[col] = [dtype, count, missing, unique, mode, round(min, roundto), round(q1, roundto),
                             round(median, roundto), round(q3, roundto), round(max, roundto), round(mean, roundto),
                             round(std, roundto), round(skew, roundto), round(kurt, roundto)]
    else:

      df_results.loc[col] = [dtype, count, missing, unique, mode, "-", "-", "-", "-", "-", "-", "-", "-", "-"]

  return df_results

In [2]:
# import necessary csv files and convert to dataframes
import pandas as pd
import sqlite3


dfM = pd.read_csv('movies_titles.csv')
print(dfM.shape)
dfM.head()

(8508, 42)


,show_id,type,title,director,cast,country,release_year,rating,duration,description,...,Language TV Shows,Musicals,Nature TV,Reality TV,Spirituality,TV Action,TV Comedies,TV Dramas,Talk Shows TV Comedies,Thrillers
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,"Michael Hilow, Ana Hoffman, Dick Johnson, Kirs...",United States,2020,PG-13,90 min,As her father nears the end of his life filmma...,...,0,0,0,0,0,0,0,0,0,0
1,s2,TV Show,Blood & Water,NaN,Ama Qamata Khosi Ngema Gail Mabalane Thabang M...,South Africa,2021,TV-MA,2 Seasons,After crossing paths at a party a Cape Town te...,...,0,0,0,0,0,0,0,1,0,0
2,s3,TV Show,Ganglands,Julien Leclercq,Sami Bouajila Tracy Gotoas Samuel Jouy Nabiha ...,NaN,2021,TV-MA,1 Season,To protect his family from a powerful drug lor...,...,0,0,0,0,0,1,0,0,0,0
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021,TV-MA,1 Season,Feuds flirtations and toilet talk go down amon...,...,0,0,0,1,0,0,0,0,0,0
4,s5,TV Show,Kota Factory,NaN,Mayur More Jitendra Kumar Ranjan Raj Alam Khan...,India,2021,TV-MA,2 Seasons,In a city of coaching centers known to train I...,...,0,0,0,0,0,0,1,0,0,0


In [3]:
#might need to go back and look at why data might be missing instead of just dropping all of it
df = dfM.loc[:, ['show_id', 'title', 'type', 'director', 'cast', 'country', 'rating', 'description']]
df.fillna('the', inplace=True)
univariate_stats(df)


,dtype,count,missing,unique,mode,min,q1,median,q3,max,mean,std,skew,kurt
show_id,object,8508,0,8508,s1,-,-,-,-,-,-,-,-,-
title,object,8508,0,8505,15-Aug,-,-,-,-,-,-,-,-,-
type,object,8508,0,2,Movie,-,-,-,-,-,-,-,-,-
director,object,8508,0,4704,the,-,-,-,-,-,-,-,-,-
cast,object,8508,0,8118,the,-,-,-,-,-,-,-,-,-
country,object,8508,0,732,United States,-,-,-,-,-,-,-,-,-
rating,object,8508,0,15,TV-MA,-,-,-,-,-,-,-,-,-
description,object,8508,0,8476,Paranormal activity at a lush abandoned proper...,-,-,-,-,-,-,-,-,-


In [ ]:
# looking for reasons why a value might be null
#dfM.loc[pd.isnull(dfM['cast'])].head(1)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10)

# create tfidfvectorizer and remove stop words
tfidf = TfidfVectorizer(stop_words="english")

# fit/transform data to a tfdidf matrix - if making from more tahn one df column 
# you have to concatonate the columns in a way that makes sense and makes everything unique
df['cast'] = df['cast'].str.replace(' ', '', regex=False)
matrixdf = pd.DataFrame({'show_id': df['show_id'], 
                        'words': 
                        df['director'].str.replace(' ', '',regex=False) + ' ' + 
                        df['cast'].str.replace(',', ' ', regex=False) + ' ' + 
                        df['country'].str.replace(' ', '', regex=False) + ' ' + 
                        df['rating'] + ' ' + 
                        df['description']})

tfidf_matrix = tfidf.fit_transform(matrixdf['words'])
# were using the matrix form instead of the df form but this is the df form:
df_tfidf = pd.DataFrame(tfidf_matrix.T.todense(), 
                        index=tfidf.get_feature_names_out(), 
                        columns=matrixdf['words'])
#df_tfidf

In [5]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

df_results = pd.DataFrame(cosine_sim, index=matrixdf['show_id'], columns=matrixdf['show_id'])
df_results

show_id,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s34,s35,s36,s37,s38,s39,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49,s50,s51,...,s8758,s8759,s8760,s8761,s8762,s8763,s8764,s8765,s8766,s8767,s8768,s8769,s8770,s8771,s8772,s8773,s8774,s8775,s8776,s8777,s8778,s8779,s8780,s8781,s8782,s8783,s8784,s8785,s8786,s8787,s8788,s8789,s8790,s8791,s8792,s8793,s8794,s8795,s8796,s8797,s8798,s8799,s8800,s8801,s8802,s8803,s8804,s8805,s8806,s8807
show_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
s1,1.000000,0.000000,0.000000,0.000000,0.010126,0.000000,0.005889,0.000000,0.025045,0.033749,0.000000,0.000000,0.000000,0.006634,0.000000,0.013903,0.007973,0.000000,0.000000,0.000000,0.000000,0.000000,0.020808,0.000000,0.016160,0.000000,0.023901,0.023049,0.032912,0.020631,0.010127,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017405,0.000000,0.004135,0.009266,0.010706,0.009555,0.022974,0.005655,0.000000,0.000000,0.003785,0.000000,0.007134,...,0.024750,0.006935,0.006282,0.020643,0.000000,0.010898,0.041225,0.019883,0.0,0.021489,0.022240,0.0,0.008122,0.020067,0.000000,0.000000,0.005334,0.000000,0.007668,0.006501,0.006467,0.032588,0.006242,0.004211,0.000000,0.038559,0.000000,0.000000,0.000000,0.000000,0.009687,0.023949,0.006347,0.004190,0.004695,0.0,0.029768,0.000000,0.000000,0.005952,0.000000,0.000000,0.000000,0.006749,0.000000,0.004091,0.000000,0.004233,0.019908,0.000000
s2,0.000000,1.000000,0.006822,0.006966,0.006648,0.030293,0.040327,0.006013,0.001708,0.000000,0.005832,0.007152,0.007450,0.024630,0.005721,0.006478,0.005235,0.006745,0.001795,0.006969,0.001333,0.001634,0.001573,0.057534,0.001684,0.002061,0.001777,0.000000,0.000000,0.000000,0.001746,0.044529,0.004839,0.001958,0.007531,0.005719,0.001492,0.000000,0.001855,0.028136,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005820,0.039648,0.006737,0.001965,...,0.000000,0.001910,0.001730,0.006449,0.001774,0.007155,0.001317,0.000000,0.0,0.000000,0.000000,0.0,0.005333,0.002092,0.002089,0.002005,0.001469,0.001802,0.028466,0.000000,0.001781,0.001781,0.001719,0.001774,0.001546,0.084718,0.001452,0.002119,0.002037,0.022395,0.001613,0.030968,0.001748,0.000000,0.000000,0.0,0.000000,0.001573,0.001699,0.001639,0.027122,0.029811,0.089847,0.001859,0.028692,0.000000,0.037147,0.000000,0.000000,0.001814
s3,0.000000,0.006822,1.000000,0.006928,0.006612,0.006159,0.000000,0.005980,0.001699,0.000000,0.005800,0.007113,0.024756,0.001817,0.005689,0.006442,0.005207,0.039017,0.042057,0.006931,0.001326,0.001625,0.001565,0.001604,0.001675,0.002050,0.001768,0.000000,0.053200,0.000000,0.001736,0.006121,0.033319,0.001947,0.007490,0.033080,0.001483,0.000000,0.001845,0.001732,0.000000,0.080011,0.000000,0.037994,0.000000,0.000000,0.005788,0.000000,0.006700,0.031691,...,0.000000,0.001900,0.001721,0.006414,0.001764,0.007116,0.021239,0.000000,0.0,0.000000,0.000000,0.0,0.005304,0.002080,0.020599,0.001994,0.001461,0.001793,0.020828,0.015878,0.001772,0.001772,0.001710,0.001764,0.001538,0.000000,0.001444,0.020901,0.002026,0.016386,0.015911,0.020669,0.001738,0.000000,0.000000,0.0,0.000000,0.001565,0.001690,0.001630,0.001670,0.006061,0.001843,0.001849,0.005834,0.000000,0.002335,0.000000,0.000000,0.017893
s4,0.000000,0.006966,0.006928,1.000000,0.006752,0.006290,0.000000,0.006106,0.001735,0.000000,0.005922,0.007264,0.007566,0.017616,0.005810,0.006578,0.005317,0.006850,0.017304,0.050969,0.001354,0.015752,0.001598,0.001638,0.001711,0.002093,0.001805,0.000000,0.000000,0.000000,0.001773,0.006251,0.004914,0.001988,0.007649,0.005808,0.001515,0.000000,0.001884,0.001769,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005910,0.038244,0.049267,0.001996,...,0.053371,0.001940,0.001757,0.006549,0.001802,0.007267,0.001337,0.000000,0.0,0.000000,0.000000,0.0,0.005416,0.002124,0.002121,0.002036,0.001492,0.001831,0.002145,0.000000,0.017173,0.032536,0.016574,0.017103,0.001570,0.000000,0.001474,0.038710,0.002069,0.01

In [ ]:
# Flatten the similarity matrix
df_flat = df_results.reset_index().melt(
    id_vars='show_id',
    var_name='recommended_show_id',
    value_name='similarity_score'
)

# Remove self-comparisons
df_flat = df_flat[df_flat['show_id'] != df_flat['recommended_show_id']]

# Get top 10 recommendations per show_id
df_top = (
    df_flat.sort_values(['show_id', 'similarity_score'], ascending=[True, False])
           .groupby('show_id')
           .head(10)
)

# Convert top recommended IDs to a comma-separated string
df_grouped = (
    df_top.groupby('show_id')['recommended_show_id']
          .apply(lambda x: ','.join(x.astype(str)))
          .reset_index(name='recommended_show_ids')
)

In [10]:
df_grouped.shape

(8508, 2)

In [12]:
# store recommendations for all posts and store them in a sqlite file
# Step 4: Export to SQLite (requires sqlite3 connection)
import sqlite3
# Step 5: Save to SQLite
conn = sqlite3.connect("show_content_filter.sqlite")
df_grouped.to_sql("movie_recommendations", conn, if_exists="replace", index=False)
conn.close()